In [1]:
!pip install -U "setfit[absa]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.0 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 1.0.1
    Uninstalling blis-1.0.1:
      Successfully uninstalled blis-1.0.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.2
    Uninstalling thinc-8.3.2:
      Successfully uninstalled thinc-8.3.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.2
    Uninstalling spacy-3.8.2:
      Successfully uninstalled spacy-3.8.2


In [2]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-lg
    Found existing installation: en_core_web_lg 3.8.0
    Uninstalling en_core_web_lg-3.8.0:
      Successfully uninstalled en_core_web_lg-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
from setfit import AbsaTrainer, TrainingArguments, AbsaModel
from datasets import load_dataset
from sklearn.metrics import f1_score

In [4]:
dataset = load_dataset("ronenlap/SetFitAbsa_FiQA")
train_ds = dataset["train"]
test_ds = dataset["test"]

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


README.md:   0%|          | 0.00/519 [00:00<?, ?B/s]

(…)-00000-of-00001-fd8d89bc25b60b32.parquet:   0%|          | 0.00/41.8k [00:00<?, ?B/s]

(…)-00000-of-00001-4a43e15cd8857d20.parquet:   0%|          | 0.00/17.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/670 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/235 [00:00<?, ? examples/s]

In [5]:
k = 24
seed = 35

In [6]:
experiment_ds = train_ds.shuffle(seed=seed).select(range(k))

In [7]:
model = AbsaModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
)

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [8]:
import os
os.makedirs("checkpoints", exist_ok = True)

In [9]:
args = TrainingArguments(
    output_dir = "checkpoints",
    num_epochs=1,
    batch_size=4,
    num_iterations=20,
    save_strategy="no",
    report_to="none",
    logging_strategy = "no",
)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'aspect', 'polarity', 'ordinal'],
        num_rows: 670
    })
    test: Dataset({
        features: ['sentence', 'aspect', 'polarity', 'ordinal'],
        num_rows: 235
    })
})

In [11]:
trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=experiment_ds,  # if you want to train over the entire train set change experiment_ds to train_ds
    column_mapping={
        "sentence": "text",
        "aspect": "span",
        "polarity": "label",
        "ordinal": "ordinal",
    },
)

/opt/conda/lib/python3.10/site-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Map:   0%|          | 0/97 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
trainer.train()

***** Running training *****
  Num unique pairs = 3880
  Batch size = 4
  Num epochs = 1
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bhuvanesh_ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241107_052807-3lef7ilw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run earthy-sponge-4
wandb: ⭐️ View project at https://wandb.ai/bhuvanesh_ai/sentence-transformers
wandb: 🚀 View run at https://wandb.ai/bhuvanesh_ai/sentence-transformers/runs/3lef7ilw


Step,Training Loss


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

***** Running training *****
  Num unique pairs = 960
  Batch size = 4
  Num epochs = 1


Step,Training Loss


/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
test_ds = test_ds.rename_columns({"sentence": "text","aspect": "span"})


In [15]:
output  = model.predict(test_ds) # a new column which holds the predicted polarity, "pred_polarity", is added to the dataset
output

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'span', 'polarity', 'ordinal', 'pred_polarity'],
    num_rows: 235
})

In [16]:
weighted_f1_score = f1_score(output["polarity"], output["pred_polarity"], average="weighted")
print(f"weighted_f1_score: {weighted_f1_score}")

weighted_f1_score: 0.7719545581926751


In [17]:
sentences = [
    "#Tesla: Model X Recall Adds To Reliability Issues $TSLA https://t.co/jVXQ4DoXnP",
    "$CIEN seems to have broken out of a major horizontal resistance. Targets $14.35.",
    "$AAPL I am big OUT from this. seems its falling towards 530.. :(",
]

model.predict(sentences)

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'span': 'Tesla', 'polarity': 'Negative'}],
 [{'span': 'CIEN', 'polarity': 'Positive'}],
 [{'span': 'AAPL', 'polarity': 'Positive'}]]

In [18]:
# Save each component as a TensorFlow model (if applicable)
# model.aspect_extractor.save('aspect_extractor_tf')
# model.aspect_model.save('aspect_model_tf')
# model.polarity_model.save('polarity_model_tf')


In [19]:
model.save_pretrained(
    "checkpoints/setfit-absa-model-aspect",
    "checkpoints/setfit-absa-model-polarity",
)

In [20]:
# !ls ./checkpoints

In [21]:
# model = AbsaModel.from_pretrained(
#     "./checkpoints/setfit-absa-model-aspect",
#     "./checkpoints/setfit-absa-model-polarity",
#     spacy_model="en_core_web_lg",
# )

In [22]:
# preds = model.predict([
#     "Best pizza outside of Italy and really tasty.",
#     "The food variations are great and the prices are absolutely fair.",
#     "Unfortunately, you have to expect some waiting time and get a note with a waiting number if it should be very full."
# ])
# print(preds)